In [1]:
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle
import os
import pandas as pd
import cv2
import random
import shutil

In [2]:

print("loading model");
model = load_model("C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_vgg-ckpnt.model");
lb = pickle.loads(open("C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\budioes.pickle", "rb").read());

test_benchmark = "C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\test_benchmark";

y_pred = []
y_true = []
exts = [".JPG",".PNG",".TIFF",".BMP",".JPEG"]

results = pd.DataFrame(columns=['path','new_path','predicted_result','expected_result','status'])

path = "C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\val"

names = {}

for x in [os.path.join(path,y) for y in os.listdir(path) if os.path.isdir(os.path.join(path,y))]:
    
    files = sorted(list([os.path.join(x,name) for name in os.listdir(x) if os.path.splitext(name)[1].upper() in exts]));
    count = 0;

    for file in files:
        status = 'FALSE';
        fname = os.path.basename(file);
        truelabel =  os.path.basename(os.path.dirname(file));
        
        truelabel = truelabel.upper();
        
        image = cv2.imread(file);
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB);
        image = cv2.resize(image, (224,224));

        image = img_to_array(image);
        image = np.expand_dims(image, axis = 0); #para incluir uma dimensão na frente da imagem, que seria o batch size
        
        #classificando
        classea = model.predict(image)[0]; #pega a primeira (e unica) ocorrência na dimensão do batch size
        idx = np.argmax(classea);     
        
        label = lb.classes_[idx];
        label = label.upper();
        y_pred.append(label);
        y_true.append(truelabel);
        
        if (label == truelabel):
            count += 1;
            status = 'TRUE'
 
        #Adicionando resultados em um dataframe
        new_name = str(random.randint(100000,999999))
        while new_name in names.keys():
            new_name = str(random.randint(100000,999999))
        
        names[new_name] = file
        extension = os.path.splitext(file)[1]
        results.loc[len(results.index)] = [file, os.path.join(test_benchmark,new_name + extension), label, truelabel, status]
        shutil.copy(file, os.path.join(test_benchmark,new_name + extension))
#        
#        print(label);
#        lbl = "{}: {:.2f}%".format(label, classea[idx]*100);
#        print(lbl);
    
    #Exportar matriz de confusão para excel com a acurácia de cada classe
    #Calcular F1, recall
    print("class:", truelabel)
    print("number of correct predictions: ", count);
    print("number of wrong predictions: ", len(files) - count);
    
print(confusion_matrix(y_true, y_pred));


loading model
class: SCARUS TRISPINOSUS_ADT
number of correct predictions:  38
number of wrong predictions:  0
class: SCARUS ZELINDAE_IP
number of correct predictions:  29
number of wrong predictions:  4
class: SPARISOMA AXILLARE_IP
number of correct predictions:  45
number of wrong predictions:  6
[[38  0  0]
 [ 3 29  1]
 [ 5  1 45]]


In [3]:
results.to_excel('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\resultado_vgg.xlsx', index=0)